In [ ]:
pip install pandas numpy scikit-learn flask gunicorn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset (using the Titanic dataset as an example)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Preprocess data
df = df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

# Encode categorical features
df = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)

# Define features and target
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


In [ ]:
import joblib

# Save the model and scaler
joblib.dump(model, "titanic_model.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Model and scaler saved!")


In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load the model and scaler
model = joblib.load("titanic_model.pkl")
scaler = joblib.load("scaler.pkl")

# Create Flask app
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()  # Expecting JSON input
    features = np.array(data["features"]).reshape(1, -1)
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)
    return jsonify({"prediction": int(prediction[0])})

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
curl -X POST -H "Content-Type: application/json" \
    -d '{"features": [3, 35, 0, 0, 8.05, 0, 1, 0, 0]}' \
    http://127.0.0.1:5000/predict


In [ ]:
gunicorn --bind 0.0.0.0:5000 app:app
